In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, Dense, BatchNormalization, Softmax, GlobalAveragePooling2D

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
learning_rate = 0.0002
img_size = 150
batch_size = 16

In [ ]:
#cd '/content/gdrive/My Drive/TensorFlow_Training_13th'
cd '/content/gdrive/My Drive/practice'

In [ ]:
cur_dir = os.getcwd()

train_dir = os.path.join(cur_dir, 'simpsons_train')
val_dir = os.path.join(cur_dir, 'simpsons_val')

n_class = len(os.listdir(train_dir))
print("This Dataset has {} classes".format(n_class))

n_train = 0
n_val = 0
for _, _, files in os.walk(train_dir):
    if files:
        n_train += len(files)
for _, _, files in os.walk(val_dir):
    if files:
        n_val += len(files)
        
print("Train Dataset has {} images".format(n_train))
print("Validation Dataset has {} images".format(n_val))

In [ ]:
datagen = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
sample_dir = os.listdir(train_dir)[0]
fnames = sorted([os.path.join(train_dir, sample_dir, fname) for fname in os.listdir(
                                                            os.path.join(train_dir, sample_dir))])

In [ ]:
num_org_imgs = 5
num_iter = 20
img_paths = fnames[:num_org_imgs]

In [ ]:
img_list = []
for img_path in img_paths:
    img = image.load_img(img_path, target_size=(img_size, img_size))
    x = image.img_to_array(img)
    img_list.append(x)

img_list = np.array(img_list)
img_list.shape


In [ ]:
row = num_iter // num_org_imgs
col = num_org_imgs
i = 0
plt.figure(figsize=(col*4, row*4))
for batch in datagen.flow(img_list, batch_size=1):
    plt.subplot(row, col, i+1)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i == num_iter:
        break

plt.show()

In [ ]:
model = models.Sequential()
model.add(Conv2D(32, 3, input_shape=(img_size, img_size, 3)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D())
model.add(Conv2D(128, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(256, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D())
model.add(Conv2D(512, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(512, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(n_class))
model.add(BatchNormalization())
model.add(Softmax())
model.summary()

In [ ]:
model.compile(optimizers.Adam(learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_size, img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir,
                                               target_size=(img_size, img_size),
                                               batch_size=batch_size,
                                               class_mode='categorical')

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=n_train//batch_size,
                             epochs=10,
                             validation_data=val_generator,
                             validation_steps=n_val//batch_size)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()